In [1]:
import requests

# List of Dropbox links and corresponding output filenames
files_to_download = [
    ("https://www.dropbox.com/scl/fi/00bsx7padbmmgozegdixh/coarsened_eta_output_GCOM.nc?rlkey=75tzzsaanoagf6gu83s09w4g7&st=jc5c2eba&dl=1", "coarsened_eta_output_GCOM.nc"),
    ("https://www.dropbox.com/scl/fi/i90p1hazy6ns5q74me3vh/coarsened_eta_output_Capella.nc?rlkey=hzq8coi5nu7oeasb9t1gxvif8&st=mnuh4cy3&dl=1", "coarsened_eta_output_Capella.nc"),
    ("https://www.dropbox.com/scl/fi/k1d3tjbo2d4s5d5zdnpiv/eta0_resampled_to_match_coarsened_grid.nc?rlkey=4tcwsppna2pu7q8zipjuvqlkb&st=w2hqtzv2&dl=1", "eta0_resampled_to_match_coarsened_grid.nc"),
    ("https://www.dropbox.com/scl/fi/7ie9jhj5d5m00y96cad8l/efficiency_snow_cover.nc?rlkey=44vgvzwixr92cmkhycqivz8i7&st=ig665qqo&dl=1", "efficiency_snow_cover.nc"),
    ("https://www.dropbox.com/scl/fi/op0nflt34tv4pd8fahl1r/efficiency_resolution_layer.nc?rlkey=0u1qa7d9xi3atvden4qa2sxnv&st=3g69r43a&dl=1", "efficiency_resolution_layer.nc")
]

for url, output_file in files_to_download:
    try:
        # Send a GET request to the URL
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for HTTP errors

        # Write the content to a file in chunks to avoid memory issues
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Download completed successfully! File saved as: {output_file}")
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {output_file}: {http_err}")
    except Exception as err:
        print(f"An error occurred for {output_file}: {err}")


Download completed successfully! File saved as: coarsened_eta_output_GCOM.nc
Download completed successfully! File saved as: coarsened_eta_output_Capella.nc
Download completed successfully! File saved as: eta0_resampled_to_match_coarsened_grid.nc
Download completed successfully! File saved as: efficiency_snow_cover.nc
Download completed successfully! File saved as: efficiency_resolution_layer.nc


In [2]:
import xarray as xr
import rioxarray

def process_satellite_data(coarsened_eta_file, eta0_resampled_file, snow_cover_file, resolution_layer_file, weights, output_filename):
    """
    Function to process satellite data by applying weighted multiplication for each dataset.
    
    Parameters:
    - coarsened_eta_file (str): Filepath for coarsened ETA dataset (specific to each satellite).
    - eta0_resampled_file (str): Filepath for resampled eta0 dataset (shared between satellites).
    - snow_cover_file (str): Filepath for snow cover dataset (shared between satellites).
    - resolution_layer_file (str): Filepath for resolution layer dataset (shared between satellites).
    - weights (dict): Dictionary containing the weights for each dataset (e.g., 'eta0', 'snow_cover', 'resolution_layer').
    - output_filename (str): The name of the output NetCDF file.

    Returns:
    - final_combined_ds (xr.Dataset): The final combined dataset.
    """
    
    # Load the coarsened eta dataset (specific for each satellite)
    coarsened_eta_ds = xr.open_dataset(coarsened_eta_file).drop_vars('spatial_ref', errors='ignore')
    
    # Ensure the CRS is set for the coarsened eta dataset
    if not coarsened_eta_ds.rio.crs:
        coarsened_eta_ds = coarsened_eta_ds.rio.write_crs("EPSG:4326")

    # Select the second time step (timestep=1)
    coarsened_eta_timestep = coarsened_eta_ds['coarsened_eta_result'].isel(time=1)

    # Scale the selected time step by the weight
    coarsened_eta_scaled = coarsened_eta_timestep * weights['coarsened_eta']

    # Load the other datasets
    eta0_resampled_ds = xr.open_dataset(eta0_resampled_file).drop_vars('spatial_ref', errors='ignore')
    snow_cover_ds = xr.open_dataset(snow_cover_file).drop_vars('spatial_ref', errors='ignore')
    resolution_layer_ds = xr.open_dataset(resolution_layer_file).drop_vars('spatial_ref', errors='ignore')

    # Ensure CRS is set for all datasets (eta0, snow cover, and resolution)
    datasets = [
        (eta0_resampled_ds, 'eta0'),
        (snow_cover_ds, 'Day_CMG_Snow_Cover'),
        (resolution_layer_ds, 'Monthly_Resolution_Abs')
    ]
    
    for dataset, var_name in datasets:
        if not dataset[var_name].rio.crs:
            dataset[var_name] = dataset[var_name].rio.write_crs("EPSG:4326")

    # Select the corresponding timestep for eta0_resampled (time=1)
    eta0_scaled = eta0_resampled_ds['eta0'].isel(time=1) * weights['eta0']

    # Extract the corresponding date from the coarsened_eta_ds dataset
    corresponding_date = coarsened_eta_ds['time'].isel(time=1).values

    # Match the date in snow cover dataset
    matching_snow_cover = snow_cover_ds.sel(time=corresponding_date, method='nearest')
    snow_cover_scaled = matching_snow_cover['Day_CMG_Snow_Cover'] * weights['snow_cover']

    # Scale the resolution layer (no need for time dimension matching here)
    resolution_layer_scaled = resolution_layer_ds['Monthly_Resolution_Abs'] * weights['resolution_layer']

    # Reproject all datasets to match the coarsened_eta_scaled dataset
    eta0_resampled = eta0_scaled.rio.reproject_match(coarsened_eta_scaled)
    snow_cover_resampled = snow_cover_scaled.rio.reproject_match(coarsened_eta_scaled)
    resolution_layer_resampled = resolution_layer_scaled.rio.reproject_match(coarsened_eta_scaled)

    # Multiply the datasets element-wise (NaNs will be preserved)
    final_eta_combined = eta0_resampled * coarsened_eta_scaled * snow_cover_resampled * resolution_layer_resampled

    # Convert the result to a new dataset
    final_combined_ds = final_eta_combined.to_dataset(name='final_eta_result')

    # Save the final combined dataset to a new NetCDF file
    final_combined_ds.to_netcdf(output_filename)

    print(f"Weighted multiplication complete. Final result saved to '{output_filename}'.")

    return final_combined_ds

# Filepaths shared between both satellites
eta0_resampled_file = 'eta0_resampled_to_match_coarsened_grid.nc'
snow_cover_file = 'efficiency_snow_cover.nc'
resolution_layer_file = 'efficiency_resolution_layer.nc'

# Weights for each dataset (these apply to both Capella and GCOM)
weights = {
    'eta0': 0.2,  # Weight for eta0
    'coarsened_eta': 1,  # Weight for coarsened eta
    'snow_cover': 0.2,  # Weight for snow cover dataset
    'resolution_layer': 0.4  # Weight for resolution layer
}

# Processing GCOM
gcom_result = process_satellite_data(
    coarsened_eta_file='coarsened_eta_output_GCOM.nc', 
    eta0_resampled_file=eta0_resampled_file,
    snow_cover_file=snow_cover_file,
    resolution_layer_file=resolution_layer_file,
    weights=weights,
    output_filename='final_eta_snow_cover_output_GCOM.nc'
)

# Processing Capella
capella_result = process_satellite_data(
    coarsened_eta_file='coarsened_eta_output_Capella.nc', 
    eta0_resampled_file=eta0_resampled_file,
    snow_cover_file=snow_cover_file,
    resolution_layer_file=resolution_layer_file,
    weights=weights,
    output_filename='final_eta_snow_cover_output_Capella.nc'
)


Weighted multiplication complete. Final result saved to 'final_eta_snow_cover_output_GCOM.nc'.
Weighted multiplication complete. Final result saved to 'final_eta_snow_cover_output_Capella.nc'.
